In [1]:
import arcpy
import os
import time
import multiprocessing

def is_valid_location(facility, network_dataset_extent):
    """
    Check if the facility has a valid location within the network dataset extent.
    """
    if facility is None:
        return False
    # Check if the facility point is outside the extent of the network dataset
    return not network_dataset_extent.disjoint(facility)

def generate_service_area(mode, field_name, output_file_name, facilities):
    # Record the start time
    start_time = time.time()

    # Set the workspace and overwrite output
    workspace = fr"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"
    arcpy.env.workspace = workspace
    arcpy.env.overwriteOutput = True

    # Specify the network dataset
    network_dataset = fr"E:\Space-Time-Prism-Model\PwoD STP\MM_NetworkDataset_03092022.gdb\NetworkDataset\NetworkDataset_ND"
    travel_mode = mode  # Set the travel mode based on the passed mode parameter

    # Output directory path
    output_dir = rf"E:\Space-Time-Prism-Model\PwoD STP\ServiceArea_Tbig_Tsm\SerArea_{mode}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    # Create an empty feature class to store all service areas
    service_area_fc_name = f'ServiceArea_{mode}_{field_name}'
    service_area_fc = arcpy.CreateFeatureclass_management(workspace, service_area_fc_name, 'POLYGON').getOutput(0)

    # Define the coordinate system for the ServiceArea feature class
    sr = arcpy.SpatialReference("NAD 1983 UTM Zone 12N")
    arcpy.DefineProjection_management(service_area_fc, sr)

    # Add fields to store the 'hptripid' and field_name in the ServiceArea feature class
    arcpy.AddField_management(service_area_fc, "hptripid", "TEXT")
    arcpy.AddField_management(service_area_fc, "travel_time_budget", "DOUBLE")

    # Get the extent of the network dataset
    network_dataset_extent = arcpy.Describe(network_dataset).extent
    invalid_facilities = []  # List to keep track of facilities with invalid locations

    # Iterate through each person in the facilities feature class
    with arcpy.da.SearchCursor(facilities, ['hptripid', field_name, 'SHAPE@']) as cursor:
        for row in cursor:
            id, travel_time_budget, facility_shape = row = row

            # Validate the location of the facility
            if not is_valid_location(facility_shape, network_dataset_extent):
                print(f"Invalid location for facility {id}. Skipping.")
                invalid_facilities.append(id)
                continue
            # Check if travel_time_budget is greater than zero
            if travel_time_budget <= 0:
                print(f"Skipping facility {id} with invalid travel_time_budget: {travel_time_budget}")
                continue

            # Create a new service area analysis layer with a unique name
            layer_name = f"Service_Area_Layer_{id}"
            try:
                if arcpy.Exists(layer_name):
                    arcpy.Delete_management(layer_name)
                result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network_dataset, layer_name, travel_mode, 
                                                                    "FROM_FACILITIES", [travel_time_budget])
                service_area_layer = result_object.getOutput(0)
            except Exception as e:
                print(f"Error creating service area layer for facility {id}: {e}")
                continue  # Skip to the next facility

            # Add the current facility to the service area layer
            arcpy.management.MakeFeatureLayer(facilities, "current_facility_layer", f"hptripid = '{id}'")
            field_mappings = arcpy.na.NAClassFieldMappings(service_area_layer, "Facilities")
            field_mappings["Name"].mappedFieldName = "hptripid"
            arcpy.na.AddLocations(service_area_layer, "Facilities", "current_facility_layer", field_mappings)

            # Solve the service area
            arcpy.na.Solve(service_area_layer)

            # Check if the Polygons sublayer is available
            sublayers = {sublayer.name: sublayer for sublayer in service_area_layer.listLayers()}
            if "Polygons" in sublayers:
                polygons_sublayer = sublayers["Polygons"]
                arcpy.AddField_management(polygons_sublayer, "hptripid", "TEXT")
                arcpy.AddField_management(polygons_sublayer, "travel_time_budget", "DOUBLE")

                # Populate 'hptripid' and 'travel_time_budget' field in the Polygons sublayer
                with arcpy.da.UpdateCursor(polygons_sublayer, ["hptripid", "travel_time_budget"]) as update_cursor:
                    for update_row in update_cursor:
                        update_row[0], update_row[1] = id, travel_time_budget
                        update_cursor.updateRow(update_row)

                # Append the service area polygons to the ServiceArea feature class
                arcpy.management.Append(polygons_sublayer, service_area_fc, 'NO_TEST')
                arcpy.management.Delete(layer_name)
            else:
                print(f"No Polygons sublayer found for {mode} with {field_name} at {id}.")
                continue  # Skip to the next iteration

    # Save the updated feature class as a shapefile in the specified output directory
    output_file_path = os.path.join(output_dir, output_file_name)
    arcpy.CopyFeatures_management(service_area_fc, output_file_path)

    # Report invalid facilities
    if invalid_facilities:
        print(f"Skipped the following facilities due to invalid locations: {invalid_facilities}")

    # Calculate processing time
    print(f"The script took {time.time() - start_time} seconds to complete for {mode} mode with {field_name}.")

def worker(mode, field_name, output_file_name, facilities):
    try:
        generate_service_area(mode, field_name, output_file_name, facilities)
    except Exception as e:
        print(f"Error in processing {mode}, {field_name}: {e}")

def Model():
    modes = ['Bike'] # ['Bike', 'Walk', 'Transit', 'Drive']
    tasks = []

    for mode in modes:
        orig_facilities = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\origin_{mode}_filtered.shp"
        dest_facilities = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\destn_{mode}_filtered.shp"

        # Prepare tasks for the pool
        tasks.append((mode, 'Serv_Tbig', f'{mode}_Orig_serTbig.shp', orig_facilities))
        tasks.append((mode, 'Serv_Tsm', f'{mode}_Orig_serTsm.shp', orig_facilities))
        tasks.append((mode, 'Serv_Tbig', f'{mode}_Destn_SerTbig.shp', dest_facilities))
        tasks.append((mode, 'Serv_Tsm', f'{mode}_Destn_SerTsm.shp', dest_facilities))

    # Create a pool of worker processes
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        # Execute the tasks
        pool.starmap(worker, tasks)

if __name__ == '__main__':
    Model()